In [42]:
import numpy as np
from mvpa2.suite import *
import os.path as op
import sklearn
import seaborn as sns
from pywt import wavedecn
from scipy import stats
from mpl_toolkits import mplot3d
from scipy.io import loadmat
from tqdm import tqdm
from scipy.ndimage.interpolation import shift

In [14]:
subj = 's1'
run_lst = np.loadtxt(subj +'/'+'short_run_list.txt',dtype = str)
run_lst

array(['01', '03', '04', '05', '06', '08', '09', '10', '11', '12', '13',
       '15', '16', '18', '19', '20'], dtype='|S2')

In [18]:
bold_fname = []

for i in run_lst:
    all_runs= (subj+'/'+subj+'.nii/'+i+'.nii')
    bold_fname.append(all_runs)

bold_fname


['s1/s1.nii/01.nii',
 's1/s1.nii/03.nii',
 's1/s1.nii/04.nii',
 's1/s1.nii/05.nii',
 's1/s1.nii/06.nii',
 's1/s1.nii/08.nii',
 's1/s1.nii/09.nii',
 's1/s1.nii/10.nii',
 's1/s1.nii/11.nii',
 's1/s1.nii/12.nii',
 's1/s1.nii/13.nii',
 's1/s1.nii/15.nii',
 's1/s1.nii/16.nii',
 's1/s1.nii/18.nii',
 's1/s1.nii/19.nii',
 's1/s1.nii/20.nii']

In [26]:
#ds= fmri_dataset(bold_fname)

In [27]:
vt = fmri_dataset(subj+'/vt.nii') #load mat.file into python. 
#VT = VT['VT'] #get the value out of dic

In [30]:
vt.shape

(1, 172032)

In [33]:
ds = fmri_dataset(bold_fname, mask = subj+'/vt.nii') # mask = vt doesn't work here, because the error message 'array must be sequence' therefore, it has to be load with the file name

In [34]:
ds.shape

(512, 1848)

In [54]:
conditions=loadmat(subj+'/conds_short_tlrc.mat')

In [55]:
conditions = conditions['conds_short_tlrc']

In [56]:
print conditions [:,0:20]

[[0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1]]


In [57]:
conditions_sh2 = shift(conditions,[0,2], cval=0)

In [58]:
print conditions_sh2[:,0:20]

[[0 0 0 0 0 0 1 1 1 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0]]


In [63]:
 def convert_binary_to_multiclass(binary_conditions):
    """Convert binary representation into multiclass reprentation:
    For example: convert [[1 1 1 1 0 0 0 0]
                          [0 0 0 0 1 1 1 1]]
    to [1 1 1 1 2 2 2 2]"""
    x,y = np.where(binary_conditions)
    conditions=np.zeros(binary_conditions.shape[1])
    conditions[y]=x+1
    return conditions

In [64]:
conditions_multi = convert_binary_to_multiclass(conditions_sh2)
#conditions_multi[:123]#first run

In [67]:
print conditions_multi [0:20]

[0. 0. 0. 0. 0. 0. 1. 1. 1. 0. 0. 0. 0. 3. 3. 3. 0. 0. 0. 0.]


In [70]:
conditions_multi[0:50]

array([0., 0., 0., 0., 0., 0., 1., 1., 1., 0., 0., 0., 0., 3., 3., 3., 0.,
       0., 0., 0., 4., 4., 4., 0., 0., 0., 0., 2., 2., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 2., 2., 2., 0., 0., 0., 0., 1., 1., 1., 0., 0.])

In [75]:
runs = np.arange(0,512)/32
runs.shape

(512,)

In [77]:
ds = fmri_dataset (bold_fname, mask = subj+'/vt.nii', targets = conditions_multi, chunks = runs)
print ds.summary()

Dataset: 512x1848@float64, <sa: chunks,targets,time_coords,time_indices>, <fa: voxel_indices>, <a: imgaffine,imghdr,imgtype,mapper,voxel_dim,voxel_eldim>
stats: mean=2.06429e-07 std=7.6618 var=58.7032 min=-122.263 max=136.79

Counts of targets in each chunk:
  chunks\targets 0.0 1.0 2.0 3.0 4.0
                 --- --- --- --- ---
        0         20  3   3   3   3
        1         20  3   3   3   3
        2         20  3   3   3   3
        3         20  3   3   3   3
        4         20  3   3   3   3
        5         20  3   3   3   3
        6         20  3   3   3   3
        7         20  3   3   3   3
        8         20  3   3   3   3
        9         20  3   3   3   3
       10         20  3   3   3   3
       11         20  3   3   3   3
       12         20  3   3   3   3
       13         20  3   3   3   3
       14         20  3   3   3   3
       15         20  3   3   3   3

Summary for targets across chunks
  targets mean std min max #chunks
    0      20   0   2

In [78]:
orig_data = ds.a.mapper.reverse(ds.samples)

In [82]:
orig_vt = vt.a.mapper.reverse(vt.samples)
orig_vt.shape

(1, 64, 64, 42)

In [83]:
new_data = orig_vt * orig_data

In [85]:
new_data.shape

(512, 64, 64, 42)

In [87]:
from matplotlib.pylab import *
import dtcwt


In [89]:
var_L1=[]
var_L2=[]
var_L3=[]
var_L4=[]
var_L5=[]

trans = dtcwt.Transform3d()

    #animals = [insect1,insect2,insect3,insect4,bird1,bird2,bird3,bird4,monkey1,monkey2,monkey3,monkey4]
    #for i in range(5):
print "starting wavelet transform"

for i in tqdm(range(new_data.shape[0])): #transform all 1230 trs
    TR=new_data[i,:,:,:]
    wvt = trans.forward(TR,nlevels=5)

    # level 1:
    highpasses = wvt.highpasses[0]
    highpasses = np.ma.log(np.abs(highpasses))
    var_per_orientation = highpasses.var(axis=(0,1,2)) #0,1,2 means take var across x,y,z
    #total_var = np.mean(var_per_orientation)  
    var_L1.append(var_per_orientation)

    # level 2:
    highpasses = wvt.highpasses[1]
    highpasses = np.ma.log(np.abs(highpasses))
    var_per_orientation = highpasses.var(axis=(0,1,2))
    #total_var = np.mean(var_per_orientation)
    var_L2.append(var_per_orientation)

    # level 3:
    highpasses = wvt.highpasses[2]
    highpasses = np.ma.log(np.abs(highpasses))
    var_per_orientation = highpasses.var(axis=(0,1,2))
    #total_var = np.mean(var_per_orientation) 
    var_L3.append(var_per_orientation)

    # level 4:
    highpasses = wvt.highpasses[3]
    highpasses = np.ma.log(np.abs(highpasses))
    var_per_orientation = highpasses.var(axis=(0,1,2))
    #total_var = np.mean(var_per_orientation)  
    var_L4.append(var_per_orientation)

    # level 5:
    highpasses = wvt.highpasses[4]
    highpasses = np.ma.log(np.abs(highpasses))
    var_per_orientation = highpasses.var(axis=(0,1,2))
    #total_var = np.mean(var_per_orientation)  
    var_L5.append(var_per_orientation)

  0%|          | 0/512 [00:00<?, ?it/s]

starting wavelet transform


100%|██████████| 512/512 [14:54<00:00,  1.75s/it]
